# Analysis of Signal Smoothness for Increasing Neighborhood per Node

## Setup

In [ ]:
import sys
PROJECT_PATH = "/home/christopher_orlowicz1_vodafone_c/gershgorin"
sys.path.append(PROJECT_PATH)
%cd $PROJECT_PATH

%load_ext autoreload
%autoreload 2

In [ ]:
from tqdm import tqdm
from google.cloud import bigquery
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = "retina"
import numpy as np
import pandas as pd
import scipy

import src.db.big_query as bq
from src.db.preprocessing import Preprocessor
from src.db import nps
from src.graph.graph import Graph
from src.graph.nearest_neighbors import NearestNeighborGraph
from src.gsp import laplace_utils
import src.utils.plotting as plt_util

## Load the data

In [ ]:
# specify a timeframe to query
from_date = "2023-01-01"
to_date = "2023-01-30"

# read the CAR data (feedback score is known for all customers)
car_df = bq.join_car_nps(from_date, to_date)

prep = Preprocessor(from_date, to_date, data=car_df, verbose=False)
car_df, client_ids, adr_zips = prep.car_df, prep.client_ids, prep.adr_zips
car_df.shape

In [ ]:
s = nps.nps_signal(from_date, to_date, client_ids)

In [ ]:
smoothness = list()
n_neighbors = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
for k in tqdm(n_neighbors):
    A = NearestNeighborGraph(n_neighbors=k).build(car_df)
    # wrap graph for faster neighborhood queries
    graph = Graph(A)
    L = graph.laplacian()
    lqf = laplace_utils.norm_lap_quad_form(L, s)
    smoothness.append(lqf)

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(n_neighbors, smoothness)
plt.title("Signal smoothness for growing neighbourhood")
plt.xlabel("$k$ neighbours")
plt.ylabel("Smoothness")
plt.savefig("out/nps_smoothness_for_growing_neighbourhood.pdf")